**ROI Extraction from Video Using XML Annotations**

This script extracts Regions of Interest (ROIs) from a video using bounding box annotations provided in an XML file. It parses the XML to retrieve bounding box coordinates for each labeled object across frames, processes the video to crop these regions, and centers them on a uniform black canvas. The extracted ROIs are saved as separate output videos, one for each label, in an organized folder structure. This tool is useful for dataset preparation or focused analysis of specific objects in video frames.

In [ ]:
import cv2
import xml.etree.ElementTree as ET
import numpy as np
import os
import logging

logging.basicConfig(level=logging.INFO)

def parse_xml(xml_file):
    """
    Parse the XML file to extract ROI information.
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    rois_info = {}
    for track in root.findall('.//track'):
        label = int(track.get('label'))
        rois_info[label] = []
        for box in track.findall('box'):
            frame_number = int(box.get('frame'))
            xtl = float(box.get('xtl'))
            ytl = float(box.get('ytl'))
            xbr = float(box.get('xbr'))
            ybr = float(box.get('ybr'))
            rois_info[label].append({"frame": frame_number, "xtl": xtl, "ytl": ytl, "xbr": xbr, "ybr": ybr})
    return rois_info

def extract_rois_from_video(video_file, xml_file):
    """
    Extracts ROIs from a video using bounding box data from an XML file.
    """
    logging.info(f"Processing video file: {video_file}")
    logging.info(f"Processing XML file: {xml_file}")

    # Parse XML for ROIs
    rois_info = parse_xml(xml_file)
    cap = cv2.VideoCapture(video_file)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Determine max frame dimensions
    max_frame_dim = max(frame_width, frame_height, 800)
    black_frame_size = (max_frame_dim, max_frame_dim)

    # Prepare output folder
    base_filename = os.path.splitext(os.path.basename(video_file))[0].upper()
    output_dir = os.path.join('roi_output', base_filename)
    os.makedirs(output_dir, exist_ok=True)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Process each label in the XML data
    for label, roi_info in rois_info.items():
        output_video_path = os.path.join(output_dir, f'{label}.mp4')
        out = cv2.VideoWriter(output_video_path, fourcc, 20.0, black_frame_size)
        frame_count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            roi_info_frame = next((item for item in roi_info if item["frame"] == frame_count), None)
            if roi_info_frame:
                # Extract and center the ROI in a black frame
                roi = frame[int(roi_info_frame["ytl"]):int(roi_info_frame["ybr"]),
                            int(roi_info_frame["xtl"]):int(roi_info_frame["xbr"])]
                if roi.shape[0] > max_frame_dim or roi.shape[1] > max_frame_dim:
                    roi = cv2.resize(roi, (max_frame_dim, max_frame_dim))

                black_frame = np.zeros((max_frame_dim, max_frame_dim, 3), np.uint8)
                start_y = (max_frame_dim - roi.shape[0]) // 2
                start_x = (max_frame_dim - roi.shape[1]) // 2
                black_frame[start_y:start_y + roi.shape[0], start_x:start_x + roi.shape[1]] = roi
                out.write(black_frame)

            frame_count += 1

        out.release()
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    cap.release()
    logging.info("ROI Extraction Completed")

# Example usage

video_path = '/home/socialab/Md Rashidunnabi/Identification/New Identification/ANGOLA_Videos/2024_07_22_13_15_54_15_10_60.mp4'
xml_path = '/home/socialab/Md Rashidunnabi/Identification/New Identification/ANGOLA_XML/2024_07_22_13_15_54_15_10_60.xml'
extract_rois_from_video(video_path, xml_path)

**Batch ROI Extraction and File Management for Video and XML Pairs**

This script processes all videos in a specified directory and extracts Regions of Interest (ROIs) based on corresponding bounding box annotations from XML files. For each video-XML pair, it crops the annotated ROIs, centers them on a uniform black canvas, and saves them as individual labeled videos in an organized output directory. Once processed, the original video and its XML file are moved to a separate "processed" folder to keep the workspace clean. The script automates batch processing, making it efficient for handling large datasets of annotated video files.

In [ ]:
import cv2
import xml.etree.ElementTree as ET
import numpy as np
import os
import shutil
import logging

logging.basicConfig(level=logging.INFO)

def parse_xml(xml_file):
    """
    Parse the XML file to extract ROI information.
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    rois_info = {}
    for track in root.findall('.//track'):
        label = int(track.get('label'))
        rois_info[label] = []
        for box in track.findall('box'):
            frame_number = int(box.get('frame'))
            xtl = float(box.get('xtl'))
            ytl = float(box.get('ytl'))
            xbr = float(box.get('xbr'))
            ybr = float(box.get('ybr'))
            rois_info[label].append({"frame": frame_number, "xtl": xtl, "ytl": ytl, "xbr": xbr, "ybr": ybr})
    return rois_info

def extract_rois_from_video(video_file, xml_file, output_base_dir):
    """
    Extracts ROIs from a video using bounding box data from an XML file.
    """
    logging.info(f"Processing video file: {video_file}")
    logging.info(f"Processing XML file: {xml_file}")

    # Parse XML for ROIs
    rois_info = parse_xml(xml_file)
    cap = cv2.VideoCapture(video_file)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Determine max frame dimensions
    max_frame_dim = max(frame_width, frame_height, 800)
    black_frame_size = (max_frame_dim, max_frame_dim)

    # Prepare output folder
    base_filename = os.path.splitext(os.path.basename(video_file))[0].upper()
    output_dir = os.path.join(output_base_dir, base_filename)
    os.makedirs(output_dir, exist_ok=True)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Process each label in the XML data
    for label, roi_info in rois_info.items():
        output_video_path = os.path.join(output_dir, f'{label}.mp4')
        out = cv2.VideoWriter(output_video_path, fourcc, 20.0, black_frame_size)
        frame_count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            roi_info_frame = next((item for item in roi_info if item["frame"] == frame_count), None)
            if roi_info_frame:
                # Extract and center the ROI in a black frame
                roi = frame[int(roi_info_frame["ytl"]):int(roi_info_frame["ybr"]),
                            int(roi_info_frame["xtl"]):int(roi_info_frame["xbr"])]
                if roi.shape[0] > max_frame_dim or roi.shape[1] > max_frame_dim:
                    roi = cv2.resize(roi, (max_frame_dim, max_frame_dim))

                black_frame = np.zeros((max_frame_dim, max_frame_dim, 3), np.uint8)
                start_y = (max_frame_dim - roi.shape[0]) // 2
                start_x = (max_frame_dim - roi.shape[1]) // 2
                black_frame[start_y:start_y + roi.shape[0], start_x:start_x + roi.shape[1]] = roi
                out.write(black_frame)

            frame_count += 1

        out.release()
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    cap.release()
    logging.info(f"ROI extraction completed for {video_file}")

def process_all_videos_and_xmls(video_dir, xml_dir, output_dir, processed_dir):
    """
    Process all videos and their corresponding XMLs, then move them to a processed folder.
    """
    for video_file in os.listdir(video_dir):
        if video_file.endswith('.mp4'):
            base_name = os.path.splitext(video_file)[0]
            video_path = os.path.join(video_dir, video_file)
            xml_path = os.path.join(xml_dir, f"{base_name}.xml")

            if os.path.exists(xml_path):
                # Extract ROIs from the video
                extract_rois_from_video(video_path, xml_path, output_dir)

                # Move processed files to the processed folder
                os.makedirs(processed_dir, exist_ok=True)
                shutil.move(video_path, os.path.join(processed_dir, video_file))
                shutil.move(xml_path, os.path.join(processed_dir, f"{base_name}.xml"))
                logging.info(f"Moved {video_file} and {base_name}.xml to {processed_dir}")

            else:
                logging.warning(f"No corresponding XML found for {video_file}")

# Example usage
video_dir = '/home/socialab/Md Rashidunnabi/Identification/New Identification/ANGOLA_Videos'
xml_dir = '/home/socialab/Md Rashidunnabi/Identification/New Identification/ANGOLA_XML'
output_dir = '/home/socialab/Md Rashidunnabi/Identification/New Identification/roi_output'
processed_dir = '/home/socialab/Md Rashidunnabi/Identification/New Identification/processed_files'

process_all_videos_and_xmls(video_dir, xml_dir, output_dir, processed_dir)


**Extracting ROI from Frame 10 and Saving as Images**

This script processes a video and its corresponding XML annotation file to extract Regions of Interest (ROIs) specifically from frame 10. The ROIs are identified based on bounding box annotations provided in the XML and are saved as individual image files. Each image is named uniquely to include the subject number (extracted from the video filename) and the label of the object. This ensures consistent organization and allows for clear identification of extracted ROIs. The processed images are stored in a dedicated roi_output/photos folder for easy access, making this script particularly useful for preparing labeled datasets or visualizing annotated regions in video analysis tasks.

In [ ]:
import cv2
import xml.etree.ElementTree as ET
import os
import logging
import re

logging.basicConfig(level=logging.INFO)

def parse_xml(xml_file):
    """
    Parse the XML file to extract ROI information.
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    rois_info = {}
    for track in root.findall('.//track'):
        label = int(track.get('label'))
        rois_info[label] = []
        for box in track.findall('box'):
            frame_number = int(box.get('frame'))
            xtl = float(box.get('xtl'))
            ytl = float(box.get('ytl'))
            xbr = float(box.get('xbr'))
            ybr = float(box.get('ybr'))
            rois_info[label].append({"frame": frame_number, "xtl": xtl, "ytl": ytl, "xbr": xbr, "ybr": ybr})
    return rois_info

def extract_subject_number(video_filename):
    """
    Extracts the subject number from the video filename (e.g., '1' from '1.mp4').
    """
    match = re.search(r'(\d+)', video_filename)
    return match.group(1) if match else None

def extract_frame_10_roi_as_photo(video_file, xml_file):
    """
    Extracts the ROI from frame 10 of a video for each label in the XML file
    and saves it as an image with a unique name for each label and frame.
    """
    logging.info(f"Processing video file: {video_file}")
    logging.info(f"Processing XML file: {xml_file}")

    # Get the subject number from the video filename
    subject_number = extract_subject_number(os.path.basename(video_file))
    if subject_number is None:
        logging.error("Could not extract subject number from video filename.")
        return

    # Parse XML for ROIs
    rois_info = parse_xml(xml_file)
    cap = cv2.VideoCapture(video_file)

    # Prepare output folder with consistent naming
    output_dir = os.path.join('roi_output', 'photos')
    os.makedirs(output_dir, exist_ok=True)

    target_frame = 10  # Frame to extract

    # Process each label in the XML data
    for label, roi_info in rois_info.items():
        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret or frame_count > target_frame:
                break

            # Check if the current frame is the target frame
            if frame_count == target_frame:
                roi_info_frame = next((item for item in roi_info if item["frame"] == frame_count), None)
                if roi_info_frame:
                    # Extract the ROI
                    roi = frame[int(roi_info_frame["ytl"]):int(roi_info_frame["ybr"]),
                                int(roi_info_frame["xtl"]):int(roi_info_frame["xbr"])]

                    # Save ROI as an image with a unique name including the subject and label
                    output_image_path = os.path.join(output_dir, f'{subject_number}_label_{label}.jpg')
                    cv2.imwrite(output_image_path, roi)
                    logging.info(f"Saved ROI for subject {subject_number}, label {label}, from frame {frame_count} as {output_image_path}")

            frame_count += 1

        # Reset frame position for the next label
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    cap.release()
    logging.info("Photo Extraction Completed")

# Example usage
video_path = '/home/it/Downloads/Angola Identification-20241115T121501Z-001/Angola Identification/Sample/2024_09_19_14_00_37_5.8_10_30.mp4'
xml_path = '/home/it/Downloads/Angola Identification-20241115T121501Z-001/Angola Identification/Sample/annotations.xml'
extract_frame_10_roi_as_photo(video_path, xml_path)


**Generate a Sequentially Labeled Photo Poster**

This script creates a photo poster by arranging images from a directory into a grid layout with labels extracted from their filenames. Each image is resized to a consistent size and positioned sequentially based on its label, with customizable parameters for grid layout, padding, and font size. The script dynamically calculates the poster dimensions to accommodate the images and saves the output as a single image file. This tool is useful for organizing labeled datasets, creating visual summaries, or presenting images in a structured format.

In [ ]:
import os
from PIL import Image, ImageDraw, ImageFont
import re

# Directory containing the images
image_dir = '/home/it/Downloads/Angola Identification-20241115T121501Z-001/Angola Identification/roi_output/photos'
# Output path for the poster
output_path = '/home/it/Downloads/Angola Identification-20241115T121501Z-001/Angola Identification/roi_output/photos_Sample.png'

# Parameters
images_per_row = 10  # Number of images per row
image_size = (250, 250)  # Increased thumbnail size for each image
padding = 10  # Reduced space between images and text
font_size = 20  # Slightly larger font size for the labels

def extract_label_from_filename(filename):
    """
    Extracts the label number from a filename of the format '{subject_number}_label_{label}.jpg'.
    """
    match = re.search(r'label_(\d+)', filename)
    return int(match.group(1)) if match else None

def create_sequential_photo_poster(image_dir, output_path, images_per_row, image_size, padding, font_size):
    # Load font (using a default PIL font as fallback)
    try:
        font = ImageFont.truetype("arial.ttf", font_size)
    except IOError:
        font = ImageFont.load_default()

    # Collect and sort image paths based on label number
    image_files = [
        (extract_label_from_filename(f), os.path.join(image_dir, f))
        for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))
    ]
    image_files = sorted(image_files, key=lambda x: x[0] if x[0] is not None else float('inf'))
    image_files = [path for _, path in image_files]  # Keep only sorted paths

    num_images = len(image_files)

    # Calculate poster dimensions
    rows = (num_images + images_per_row - 1) // images_per_row
    poster_width = images_per_row * (image_size[0] + padding)
    poster_height = rows * (image_size[1] + 2 * padding + font_size)

    # Create blank poster
    poster = Image.new('RGB', (poster_width, poster_height), 'white')
    draw = ImageDraw.Draw(poster)

    # Place images in the grid with labels from filenames
    for idx, image_file in enumerate(image_files):
        row, col = divmod(idx, images_per_row)
        x = col * (image_size[0] + padding)
        y = row * (image_size[1] + 2 * padding + font_size)

        # Open, resize, and paste the image
        img = Image.open(image_file)
        img = img.resize(image_size, Image.LANCZOS)
        poster.paste(img, (x, y + font_size + padding))

        # Extract the label for display
        label = extract_label_from_filename(os.path.basename(image_file))
        label_text = str(label) if label is not None else "Unknown"

        # Center the label above the image
        text_width = draw.textbbox((0, 0), label_text, font=font)[2]
        text_x = x + (image_size[0] - text_width) // 2
        draw.text((text_x, y), label_text, fill="black", font=font)

    # Save the poster
    poster.save(output_path)
    print(f"Poster saved to {output_path}")

# Run the function
create_sequential_photo_poster(image_dir, output_path, images_per_row, image_size, padding, font_size)


Poster saved to /home/it/Downloads/Angola Identification-20241115T121501Z-001/Angola Identification/roi_output/photos_Sample.png


**Remove "Outside" Bounding Boxes from XML Annotations**

This script processes an XML file containing annotation data and removes all <box> elements marked with the attribute outside="1", indicating irrelevant or inactive bounding boxes. It ensures that the remaining annotations are clean and useful for tasks like object detection or tracking. The modified XML structure is saved to a new file while preserving the integrity of the original data format. This tool is particularly useful for refining datasets and improving the quality of annotation files in large-scale projects.

In [ ]:
import xml.etree.ElementTree as ET

def remove_outside_boxes(input_file, output_file):
    # Carrega o XML
    tree = ET.parse(input_file)
    root = tree.getroot()

    # Itera sobre todas as tags <box> com outside="1"
    for parent in root.findall(".//"):
        for box in parent.findall("box"):
            if box.get("outside") == "1":
                parent.remove(box)  # Remove do pai correto

    # Salva o XML modificado
    tree.write(output_file, encoding="utf-8", xml_declaration=True)
    print(f"Arquivo processado com sucesso. Resultado salvo em: {output_file}")

# Caminho do arquivo XML de entrada e saída

# Caminho do arquivo XML de entrada e saída
input_file = "/home/it/Downloads/Angola Identification-20241115T121501Z-001/Angola Identification/ANGOLA_Xml/2024_07_22_12_46_00_5.8_10_30.xml"
output_file = "/home/it/Downloads/Angola Identification-20241115T121501Z-001/Angola Identification/ANGOLA_Xml/2024_07_22_12_46_00_5.8_10_30_new.xml"

# Chama a função
remove_outside_boxes(input_file, output_file)


Arquivo processado com sucesso. Resultado salvo em: /home/it/Downloads/Angola Identification-20241115T121501Z-001/Angola Identification/ANGOLA_Xml/2024_07_22_12_46_00_5.8_10_30_new.xml


**Detecting Duplicate and Missing Labels in XML Annotations**

This script processes an XML annotation file to identify any duplicate or missing labels within <track> elements. It extracts all label attributes, checks for duplicates, and compares the extracted labels with an expected range (e.g., 1 to 60) to determine any missing labels. The script outputs a list of duplicate and missing labels, helping ensure the annotation dataset is complete and free of inconsistencies, making it suitable for accurate training and validation in machine learning workflows.

In [ ]:
import xml.etree.ElementTree as ET

# Load and parse the XML file
file_path = "/home/socialab/Md Rashidunnabi/Identification/New Folder/Duplicate Checking/2024_07_22_12_56_39_90_120_60.xml"
tree = ET.parse(file_path)
root = tree.getroot()

# Extract all label attributes from track tags
labels = []
for track in root.findall(".//track"):  # Find all track elements
    label = track.get("label")  # Get the 'label' attribute
    if label:
        labels.append(label)

# Find duplicate labels
duplicates = set([label for label in labels if labels.count(label) > 1])

# Define the full list of expected labels (1 to 60 as strings)
expected_labels = {str(i) for i in range(1, 61)}

# Find missing labels
found_labels = set(labels)
missing_labels = expected_labels - found_labels

# Output results
if duplicates:
    print("Duplicate labels found:", duplicates)
else:
    print("No duplicate labels found.")

if missing_labels:
    print("Missing labels:", sorted(missing_labels))
else:
    print("No missing labels.")


**Dynamic File Renaming Based on JSON Suffix Mapping**

This script dynamically renames files within directories based on a JSON mapping that specifies suffixes for each directory. It iterates through all directories in a base folder, checks if they are listed in the mapping, and appends the corresponding suffix to the filenames within those directories. This ensures consistent and automated file naming while maintaining an organized structure, making it particularly useful for handling large datasets with specific naming conventions.


> The json file can be found in the root directory




In [ ]:
import os
import json

# Paths
base_directory = '/home/socialab/Md Rashidunnabi/Identification/New Identification/roi_output'
json_file_path = '/home/socialab/Md Rashidunnabi/Identification/New Identification/directory_suffix_mapping.json'

# Load the suffix mapping from JSON
with open(json_file_path, 'r') as f:
    directory_suffix_mapping = json.load(f)

# Loop through all directories in the base directory
for directory in os.listdir(base_directory):
    directory_path = os.path.join(base_directory, directory)

    # Check if it's a directory and has a suffix in the mapping
    if os.path.isdir(directory_path) and directory in directory_suffix_mapping:
        suffix = directory_suffix_mapping[directory]

        # Rename files in the directory
        for filename in os.listdir(directory_path):
            file_path = os.path.join(directory_path, filename)
            if os.path.isfile(file_path):
                # Split the file name and extension
                name, ext = os.path.splitext(filename)

                # Add the suffix
                new_name = f"{name}{suffix}{ext}"
                new_file_path = os.path.join(directory_path, new_name)

                # Rename the file
                os.rename(file_path, new_file_path)
                print(f"Renamed: {file_path} to {new_file_path}")

print("All files have been renamed successfully.")


**Automated Folder Creation for Sequential Organization**

This script automates the creation of sequentially numbered folders, ranging from 1 to 72, within a specified base directory. It first ensures the base directory exists, then iteratively generates the numbered subfolders, avoiding errors if they already exist. This approach is useful for setting up structured directories for organizing datasets, projects, or files in a systematic and scalable manner.

In [ ]:
import os

# Set the directory where you want to create the folders
base_dir = "/home/it/Downloads/Angola Identification-20241115T121501Z-001/Angola Identification/Session 1"  # Change this to your desired path

# Create the base directory if it doesn't exist
os.makedirs(base_dir, exist_ok=True)

# Loop to create folders from 1 to 72
for i in range(1, 73):
    folder_name = str(i)
    folder_path = os.path.join(base_dir, folder_name)
    os.makedirs(folder_path, exist_ok=True)

print("Folders created successfully.")


**Organized File Distribution Based on Filename Prefix**

This script automates the organization of .mp4 files by moving them into target folders based on their filename prefixes. It traverses through the source directory, identifies files with names like 1_215.mp4, extracts the prefix (e.g., 1), and moves the files to corresponding subfolders in the target directory. If a target folder doesn’t exist, it is created dynamically. This script is particularly useful for sorting and structuring large datasets efficiently by grouping files into logically named folders.

In [ ]:
import os
import shutil

# Paths for the source and target directories
roi_output_dir = "/home/socialab/Md Rashidunnabi/Identification/New Identification/roi_output"
target_base_dir = "/home/socialab/Md Rashidunnabi/Identification/New Identification/Identification By Rashid"

# Loop through all folders in the roi_output directory
for roi_folder in os.listdir(roi_output_dir):
    roi_folder_path = os.path.join(roi_output_dir, roi_folder)

    # Ensure it's a directory
    if os.path.isdir(roi_folder_path):
        # Loop through all mp4 files in this folder
        for filename in os.listdir(roi_folder_path):
            file_path = os.path.join(roi_folder_path, filename)

            # Process only .mp4 files
            if filename.endswith(".mp4"):
                # Extract the prefix (e.g., '1' from '1_215.mp4')
                prefix = filename.split('_')[0]

                # Determine the target folder
                target_folder_path = os.path.join(target_base_dir, prefix)

                # Ensure the target folder exists
                if not os.path.exists(target_folder_path):
                    os.makedirs(target_folder_path)

                # Move the file to the target folder
                target_file_path = os.path.join(target_folder_path, filename)
                shutil.move(file_path, target_file_path)
                print(f"Moved: {file_path} to {target_file_path}")

print("All files have been moved successfully.")


**Counting Files in Sequentially Ordered Folders**

This script iterates through folders in a specified directory, sorted numerically in ascending order, to count the number of files within each folder. It generates a summary of folder names and their corresponding file counts, which is displayed in the console and optionally saved to a text file. This tool is particularly useful for auditing directory contents, ensuring consistent organization, or tracking dataset completeness.

In [ ]:
import os

# Path to the main directory
base_directory = '/home/socialab/Md Rashidunnabi/Identification/New Identification/Identification By Rashid'

# Dictionary to store the folder name and file count
folder_file_count = {}

# Iterate through all folders in the base directory in ascending order
for folder in sorted(os.listdir(base_directory), key=lambda x: int(x) if x.isdigit() else float('inf')):
    folder_path = os.path.join(base_directory, folder)
    if os.path.isdir(folder_path):  # Check if it's a directory
        # Count the number of files in the folder
        file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
        folder_file_count[folder] = file_count

# Display the results
for folder, count in folder_file_count.items():
    print(f"Folder: {folder}, Number of Files: {count}")

# Save the results to a file (optional)
output_file = os.path.join(base_directory, 'folder_file_count.txt')
with open(output_file, 'w') as f:
    for folder, count in folder_file_count.items():
        f.write(f"Folder: {folder}, Number of Files: {count}\n")

print(f"File counts saved to {output_file}")
